In [0]:
!pip install pythainlp

In [0]:
import requests
import pandas as pd
from pythainlp.tokenize import word_tokenize

In [0]:
ACCESS_TOKEN = 'EAAhtIDv5cDMBAEoZCJd81OOUKbhjVQcqLxtK6gJ1JM43pL2rtCaJByQfWo10b2dZB9OFhXleMHot7lQe7zTurcafLVS4sLNJZCMA9Ylfw5KSQYfqXYNUWtdEbquzahgzTnPojv8mPLOjxMPdAFZAZBtfxa25PEVYSZB6R5QV6Kdu71RnfyYIR9djZBU7sAF9IdgmKNPZCYZAGUlui0jS19X6v7a4Ed6cAsUHN3GhlZCvSitIusFha1Fsqq'
page_name = 'Me'
base_url = 'https://graph.facebook.com/v5.0/'+page_name
fields = 'posts{message,full_picture,created_time,shares,reactions.summary(true),comments.summary(true),permalink_url}'
url = '%s?fields=%s&access_token=%s' % (base_url,fields,ACCESS_TOKEN)
print(url)

In [0]:
content = requests.get(url).json()
list_feed = [f for f in content['posts']['data']]
df = pd.DataFrame()
while True:
    m = ''
    for f in list_feed:
        new_record = dict()
        if 'message' not in f:
            new_record['message'] = 'n/a'
            m = 'n/a'
        else:
            t_str = ' '.join(str(f['message']).split('\r'))
            m = ' '.join(t_str.split('\n'))
            tokenized_sentence = ''
            for tokens in m.split():
                if 'http' in tokens:
                    tokenized_sentence += ' '+tokens
                else:
                    tokenized_sentence += ' '+' '.join(word_tokenize(tokens))
            new_record['message'] = tokenized_sentence
            m = tokenized_sentence
        print(f['created_time'])
        if 'reactions' in f:
            print(f['reactions']['summary']['total_count'])
            new_record['reactions'] = f['reactions']['summary']['total_count']
            new_record['comments'] = f['comments']['summary']['total_count']
            new_record['created_time'] = f['created_time']
            if 'shares' in f:
                new_record['shares'] = f['shares']['count']
            else:
                new_record['shares'] = 0
            if 'permalink_url' in f:
                new_record['permalink_url'] = f['permalink_url']
            else:
                new_record['permalink_url'] = 'n/a'
        df=df.append(new_record,ignore_index=True)
    if 'posts' in content:
        if 'paging' not in content['posts'] or 'next' not in content['posts']['paging']: 
            break;        
        url = content['posts']['paging']['next']
    elif 'paging' in content:
        if 'next' not in content['paging']:
            break
        url = content['paging']['next']
    else:
        break
    print('Getting next posts')
    content = requests.get(url).json()
    list_feed = [f for f in content['data']]
df.to_csv(page_name+'me.csv')
from google.colab import files
files.download(page_name+'me.csv')

In [0]:
df=pd.read_csv('Profileme.csv')
df['created_time']=pd.to_datetime(df['created_time'])
new_df=df.groupby(pd.Grouper(key='created_time',freq='W')) 
new_df.sum()['reactions'].plot(kind='line',color=['b'],figsize=(18,12))